<a href="https://colab.research.google.com/github/watanabe3tipapa/GoogleColaboratory/blob/main/telnets_2023%E2%88%9209_07_chat_by_Go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# インターネット公告システム ホームページ

一般社団法人サイバー技術・インターネット自由研究会

> Chat機能用のクライアントです

### 準備

Goを実装します

In [1]:
!apt-get install golang


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  golang-1.18 golang-1.18-doc golang-1.18-go golang-1.18-src golang-doc
  golang-go golang-src
Suggested packages:
  bzr | brz mercurial subversion
The following NEW packages will be installed:
  golang golang-1.18 golang-1.18-doc golang-1.18-go golang-1.18-src golang-doc
  golang-go golang-src
0 upgraded, 8 newly installed, 0 to remove and 16 not upgraded.
Need to get 82.5 MB of archives.
After this operation, 437 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-doc all 1.18.1-1ubuntu1.1 [103 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-src all 1.18.1-1ubuntu1.1 [16.2 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-go amd64 1.18.1-1ubuntu1.1 [66.0 MB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/

確認

In [2]:
!go version

go version go1.18.1 linux/amd64


テスト

In [7]:
%%writefile helloworld.go
package main

import "fmt"

func main() {
    fmt.Printf("Hello World!\n")
}


Overwriting helloworld.go


In [8]:
!go run helloworld.go


Hello World!


### 実行

In [9]:
%%writefile chat.go
package main

import (
	"bufio"
	"crypto/tls"
	"fmt"
	"log"
	"os"
	"regexp"
	"strings"
	"sync"
)

var re = regexp.MustCompile("[\u001B\u009B][[\\]()#;?]*(?:(?:(?:[a-zA-Z\\d]*(?:;[a-zA-Z\\d]*)*)?\u0007)|(?:(?:\\d{1,4}(?:;\\d{0,4})*)?[\\dA-PRZcf-ntqry=><~]))")

func main() {
	config := tls.Config{Certificates: []tls.Certificate{}, InsecureSkipVerify: false}
	conn, err := tls.Dial("tcp", "koukoku.shadan.open.ad.jp:992", &config)
	if err != nil {
		log.Fatalf("client: dial: %s", err)
	}
	defer conn.Close()
	log.Println("client: connected to: ", conn.RemoteAddr())
	fmt.Fprintln(conn, "nobody")

	var wg sync.WaitGroup
	wg.Add(1)
	go func() {
		defer wg.Done()
		scanner := bufio.NewScanner(conn)
		for scanner.Scan() {
			line := strings.TrimSpace(re.ReplaceAllString(scanner.Text(), ""))
			fmt.Println(line)
		}
	}()

	wg.Add(1)
	go func() {
		defer wg.Done()
		scanner := bufio.NewScanner(os.Stdin)
		for scanner.Scan() {
			fmt.Fprintln(conn, scanner.Text())
		}
	}()

	wg.Wait()
}

Overwriting chat.go


In [10]:
!go run chat.go

2023/09/07 02:15:33 client: connected to:  103.41.63.2:992
﻿
>>チャットクライアントです。by watanabe3tipapa <<
signal: interrupt


---

^C